# Coherence-Score

In den vergangenen Teilen hast du unterschiedliche Algorithmen zur Berechnung von Topic Models kennengelernt. Du hattest das Gefühl, dass sich die NMF-Modelle besser interpretieren lassen als die LDA-Modelle. Aber stimmt das auch? Kann man das messen? Dafür gibt es den sog. *Coherence Score*, der die Güte eines Modells misst.

Die Zahl von zehn Topics ist auch vom Himmel gefallen. Das kannst du zwar als Konvention nutzen und einfach mal ausprobieren. Aber ein etwas strukturierteres Vorgehen wäre hier schon auch gut.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Nutzung von `gensim`

`gensim` ist eine auf Topic Models spezialisierte Bibliothek, die du später auch noch für Word Embeddings nutzen wirst.

Die Daten musst du für `gensim` etwas manueller vorbereiten. Du benötigst ein geschachteltes Array, in dem für jedes Dokument die Wörter bereits getrennt in Listen stehen:

In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
import regex as re
# create tokenized documents
gensim_words = [[w for w in re.split(r'[\|\#]', doc.lower()) if w not in stop_words] 
                           for doc in df["nav"]]

Damit kann dir `gensim` ein `Dictionary` erzeugen (es nummeriert dabei die Wörter):

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(gensim_words) 

Ähnlich wie bei `scikit-learn` kannst du Daten ausfiltern:

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.7)

Und schließlich die Dokumente vektorisieren. Hier nutzt du das Bag-of-words-Modell, weil du LDA als Methode nutzen möchtest:

In [ ]:
bow = [dictionary.doc2bow(doc) for doc in gensim_words]

Schließlich kannst du das Topic Model berechnen:

In [ ]:
from gensim.models import LdaModel
lda = LdaModel(corpus=bow, id2word=dictionary, num_topics=10, 
                      iterations=400, passes=20, random_state=42)

In [ ]:
lda.show_topics()

Wie du schon weißt, benötigt NMF die Vektoren im Tfidf-Format. Das kann `gensim` auch für dich ausrechnen:

In [ ]:
from gensim.models import TfidfModel
tfidf_model = TfidfModel(bow)
tfidf = tfidf_model[bow]

Jetzt kannst du von `gensim` das NMF-Modell berechnen lassen. Um ein zu `scikit-learn` vergleichbares Modell zu erreichen, kannst du die Parameter noch etwas anpassen:

In [ ]:
from gensim.models.nmf import Nmf
nmf = Nmf(corpus=tfidf, num_topics=10, id2word=dictionary, 
          w_max_iter=200,  h_max_iter=200, passes=20, random_state=42)

In [ ]:
nmf.show_topics()

Das sieht ganz ähnlich aus, gut!

## Coherence Scores

Die Berechnung der Kohärenz eines Topic Models ist ziemlich kompliziert (vielleicht auch ein Grund, wrum das in `scikit-learn` nicht implementiert ist). Dabei arbeitet de Algorithmus auch mit *Wortähnlichkeiten*, damit musst du dich aber zum Glück nicht beschäftigen.

Je höher der Score ist, desto besser ist das Modell:

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

lda_coherence = CoherenceModel(model=lda, texts=gensim_words, dictionary=dictionary, coherence='c_v')
lda_score = lda_coherence.get_coherence()
lda_score

In [ ]:
nmf_coherence = CoherenceModel(model=nmf, texts=gensim_words, dictionary=dictionary, coherence='c_v')
nmf_score = nmf_coherence.get_coherence()
nmf_score

In diesem Fall bestätigt dich der Score - die besser interpretierbare Variante hat auch den höheren Score. Das muss nicht immer so sein.

## Optimale Anzahl an Topics

Wir hatten die Anzahl der Topics relativ beliebig auf 10 gesetzt. Im Moment weißt du nicht, ob das der optimale Wert ist. Aber du kannst den *Coherence Score* für unterschiedliche viele Topics ausrechnen.

Üblicherweise wächst der Score mit  der Anzahl er Topics. Ein *lokales Maximum* ist ein guter Kandidat dafür, die richtig Anzahl von Topics gefunden zu haben. Jetzt musst du etwas Geduld haben, die Berechnung dauert ein knappe halbe Stunde.

In [ ]:
from tqdm.auto import tqdm
res = []
for num_topics in tqdm(range(5, 20)):
    lda = LdaModel(corpus=bow, id2word=dictionary, num_topics=num_topics, 
                      iterations=400, passes=20, random_state=42)
    lda_coherence = CoherenceModel(model=lda, texts=gensim_words, dictionary=dictionary, coherence='c_v')
    lda_score = lda_coherence.get_coherence()
    res.append({ "num_topics": num_topics, "score": lda_score })

In [ ]:
lda_num = pd.DataFrame(res)
lda_num.set_index("num_topics").plot()

Das Ergebnis sieht bei 13 Topics am besten aus. Kannst du das auch am besten interpretieren?

In [ ]:
lda13 = LdaModel(corpus=bow, id2word=dictionary, num_topics=13, 
                      iterations=400, passes=20, random_state=42)

In [ ]:
lda13.show_topics()

In der Tat sieht das schon deutlich besser aus, auch wenn `Apple` und `Google` immer noch sehr eng miteinander verbunden sind.

Führe die gleiche Übung nochmal mit NMF durch:

In [ ]:
res = []
for num_topics in tqdm(range(5, 20)):
    nmf = Nmf(corpus=tfidf, num_topics=num_topics, id2word=dictionary, 
              w_max_iter=200,  h_max_iter=200, passes=20, random_state=42)
    nmf_coherence = CoherenceModel(model=nmf, texts=gensim_words, dictionary=dictionary, coherence='c_v')
    nmf_score = nmf_coherence.get_coherence()
    res.append({ "num_topics": num_topics, "score": nmf_score })

In [ ]:
nmf_num = pd.DataFrame(res)
nmf_num.set_index("num_topics").plot()

Hier scheint elf eine bessere Anzahl zu sein. Probier es aus:

In [ ]:
nmf11 = Nmf(corpus=tfidf, num_topics=11, id2word=dictionary, 
            w_max_iter=200,  h_max_iter=200, passes=20, random_state=42)

In [ ]:
nmf11.show_topics()

Sieht schon etwas besser aus, allerdings taucht Google nun gar nicht mehr auf! 

## Coherence Scores zur Erklärung von Topic Models

Leider werden die Coherence Scores nicht besonders häufig verwendet. Sie können dir aber helfen, dich zwischen unterschiedlichen Topic Models zu entscheiden.

Außerdem kannst du Coherence Scores nutzen, um die optimale Menge an Topics zu finden. Allerdings solltest du dich dann auf ziemlich lange Rechenzeiten einstellen.